In [1]:
import dask_image.imread
import dask_image.ndfourier
import zarr
import matplotlib.pyplot as plt
from openslide import OpenSlide
import numpy as np

In [2]:
slide = OpenSlide('/Users/swang/Python/dask/dask images/tumor_001.tif')

In [19]:
# New Code to break sections
from numcodecs import Blosc
compressor = Blosc(cname='zstd', clevel=8, shuffle=Blosc.BITSHUFFLE)

max_y = slide.level_dimensions[0][0]
max_x = slide.level_dimensions[0][1]

In [20]:
slide.level_dimensions

((97792, 221184),
 (48896, 110592),
 (24448, 55296),
 (12224, 27648),
 (6112, 13824),
 (3056, 6912),
 (1528, 3456),
 (764, 1728),
 (382, 864),
 (191, 432))

In [21]:
max_y

97792

In [12]:
type(max_y)

int

In [22]:
layer = slide.read_region([0, 0], 0, [max_x, int(max_y/64)])
np_array = np.asarray(layer)
zarray = zarr.array(np_array, chunks=(max_y/64, max_x, 4), compressor=compressor)
zarray.info

Type,zarr.core.Array
Data type,uint8
Shape,"(1528, 221184, 4)"
Chunk shape,"(1528, 221184, 4)"
Order,C
Read-only,False
Compressor,"Blosc(cname='zstd', clevel=8, shuffle=BITSHUFFLE, blocksize=0)"
Store type,builtins.dict
No. bytes,1351876608 (1.3G)
No. bytes stored,192983986 (184.0M)
Storage ratio,7.0


In [ ]:
for i in range(1, 3):
    print(max_y*i/64)
    a = int(max_y*i/64)
    b = int(max_y/64)
    layer = slide.read_region([a, 0], 0, [max_x, b])
    np_array = np.asarray(layer)
    print(np_array.shape)
    

    z1 = zarr.array(np_array, chunks=(max_y/64, max_x, 4), compressor=compressor)
    print(z1.info)
    zarray.append(z1)

1528.0
(1528, 221184, 4)


In [18]:
zarray.info

Type,zarr.core.Array
Data type,uint8
Shape,"(1146, 55296, 4)"
Chunk shape,"(382, 55296, 4)"
Order,C
Read-only,False
Compressor,"Blosc(cname='zstd', clevel=8, shuffle=BITSHUFFLE, blocksize=0)"
Store type,builtins.dict
No. bytes,253476864 (241.7M)
No. bytes stored,33985912 (32.4M)
Storage ratio,7.5


In [98]:
storage_path = '/Users/swang/Python/dask/dask zarr files/imgzarrtest'
zarr.convenience.save(storage_path, zarray)